In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Concatenate, Input,Flatten
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import MaxNorm
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.stattools import adfuller

In [37]:
df = pd.read_csv("dispatch_data_updated.csv")

In [38]:
df = df[['ZIPCODE', 'INITIAL_CALL_TYPE', 'POLICEPRECINCT', 'INCIDENT_RESPONSE_SECONDS_QY', 'ZIPCODE_encoded', 'CALL_TYPE_encoded', 'hour', 'day', 'month', 'day_of_week',
         'INCIDENT_DATETIME','Nearest_Traffic_Vol','Distance_m','average_response_time']]

In [39]:
X = df[['ZIPCODE_encoded', 'CALL_TYPE_encoded', 'hour', 'day', 'month', 
        'day_of_week', 'POLICEPRECINCT', 'average_response_time','Distance_m','Nearest_Traffic_Vol']]

y = df['INCIDENT_RESPONSE_SECONDS_QY']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
rf = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],          
    'max_depth': [5,10],          
    'min_samples_split': [2, 5, 10],          
    'min_samples_leaf': [1, 2, 4],                
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=2, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print("Best Hyperparameters:")
print(best_params)

y_pred = best_rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")

Fitting 2 folds for each of 54 candidates, totalling 108 fits
Best Hyperparameters:
{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error (MSE): 40581.732004263315
R-squared (R²): 0.8346760590217244


In [44]:
print(f"Root Mean Squared Error (MSE): {np.sqrt(mse)}")
print(f"R-squared (R²): {r2}")

Root Mean Squared Error (MSE): 201.44908042545967
R-squared (R²): 0.8346760590217244


In [45]:
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 150.52665877543282
